In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType

from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark import SparkContext

In [3]:
conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
                     .set('spark.executor.memory', '4G')
                     .set('spark.driver.memory', '2G')
                     .set('spark.driver.maxResultSize', '4G')
                     .set('spark.sql.autoBroadcastJoinThreshold', '-1')
                     .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
                     )

In [4]:
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

In [25]:
#  def load_spark_sql(sqlContext):
#         sqlContext.read.parquet("/opt/work/data/processed/veiculos").registerTempTable("veiculos")
#         sqlContext.read.parquet("/opt/work/data/processed/linhas").registerTempTable("linhas")
#         sqlContext.read.parquet("/opt/work/data/processed/pontoslinha").registerTempTable("pontoslinha")
#         sqlContext.read.parquet("/opt/work/data/processed/tabelaveiculo").registerTempTable("tabelaveiculo")

In [26]:
# load_spark_sql(sqlContext)

In [29]:
datareferencia = '2018-08-02'

In [ ]:
#date_format(unix_timestamp('dthr', 'dd/MM/yyyy HH:mm:ss').cast('timestamp'), "yyyy-MM-dd HH:mm:ss").alias('event_timestamp')

sqlContext.read.json("/opt/work/data/raw/2018-08/veiculos") \
.select(F.col("COD_LINHA").alias("line_code"), 
        F.date_format(F.unix_timestamp('dthr', 'dd/MM/yyyy HH:mm:ss').cast('timestamp'), "yyyy-MM-dd HH:mm:ss").alias('event_timestamp'),
        F.col("LAT").cast('double').alias("latitude"),
        F.col("LON").cast('double').alias("longitude"),
        F.col("VEIC").alias("vehicle")
       )\
        .withColumn("year", F.year("event_timestamp")) \
        .withColumn("month", F.month("event_timestamp")) \
        .withColumn("day", F.dayofmonth("event_timestamp")).dropDuplicates() \
        .write.mode('overwrite') \
        .partitionBy("year", "month", "day") \
        .format("parquet") \
        .save("/opt/work/data/trusted/veiculos")

In [48]:
sqlContext.read.json("/opt/work/data/raw/2018-08/veiculos") \
.select(F.col("COD_LINHA").alias("line_code"), 
        F.date_format(F.unix_timestamp('dthr', 'dd/MM/yyyy HH:mm:ss').cast('timestamp'), "yyyy-MM-dd HH:mm:ss").alias('event_timestamp'),
        F.col("LAT").alias("latitude"),
        F.col("LON").alias("longitude"),
        F.col("VEIC").alias("vehicle")
       ).distinct().count()

11920801